<a href="https://colab.research.google.com/github/StartLate40/StockAnalysisInPython/blob/master/GB_20211202_ReLi_best_pattern_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview - daily execution

1. load similarity table from GBQ
1. load price data from gdrive
1. merge 2
1. 시초가 부터 결과

## I.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# For gbq connection
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install -q git+https://github.com/loopinf/trading_calendars.git
import pandas as pd
from trading_calendars import get_calendar
cal_krx = get_calendar('XKRX')

from pandas.tseries.offsets import CustomBusinessDay
# cbday = CustomBusinessDay(calendar=cal_krx)
cbday = CustomBusinessDay(holidays=cal_krx.adhoc_holidays)

In [ ]:
cbday + pd.Timestamp.now()

Timestamp('2021-12-07 06:22:08.825917')

In [ ]:
import itertools

In [ ]:
import pandas as pd
import pandas_gbq
date_ref = '20211202'

In [ ]:
# table_id = f"{project_id}.red_lion.pattern_{kernel_size}_{today}"
PROJECT_ID = 'dots-stock'

## II.

In [ ]:
def get_pattern(date_ref, kernel_size):
  date_ref_ = pd.Timestamp(date_ref).strftime('%Y-%m-%d')
  table_id = f"{PROJECT_ID}.red_lion.pattern_{kernel_size}_{date_ref}"
  sql = f'''
    SELECT
      *
    FROM
      `{table_id}`
    WHERE
      source_date = "{date_ref_}"
    ORDER BY
      date
    ''' 

  df = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, use_bqstorage_api=True)
  return df

In [ ]:
df_pattern_6_20211201 = get_pattern('20211201', 6)
df_pattern_6_20211202 = get_pattern('20211202', 6)
df_pattern_6_20211201.head()

Downloading: 100%|██████████| 234348/234348 [00:05<00:00, 43739.78rows/s] 


,date,Code,similarity,source_code,source_date
0,2011-01-10,000250,0.907434,340120,2021-12-01
1,2011-01-10,000390,0.963061,073190,2021-12-01
2,2011-01-10,001140,0.958982,307180,2021-12-01
3,2011-01-10,004770,0.923715,187220,2021-12-01
4,2011-01-10,004830,0.969923,063080,2021-12-01


In [ ]:
df_pattern_6_20211201_ = \
(df_pattern_6_20211201
[lambda df: df.date != df.source_date] # not same date
[lambda df: df.similarity > .95]
)

In [ ]:
print(
    df_pattern_6_20211201_.shape,
    df_pattern_6_20211201.shape
)

(105726, 5) (231594, 5)


## III.

In [ ]:
# TODO: vim yank 5 times

In [ ]:
def get_price(date_ref):
  date_ref_ = pd.Timestamp(date_ref).strftime('%Y-%m-%d')
  table_id = f"{PROJECT_ID}.red_lion.adj_price_{date_ref}"
  sql = f'''
    SELECT
      *
    FROM
      `{table_id}`
    # WHERE
    #   date = "{date_ref_}"
    ORDER BY
      date
    ''' 
  df = pandas_gbq.read_gbq(sql, project_id=PROJECT_ID, use_bqstorage_api=True)
  return df

In [ ]:
######## First time to do
# df_price = get_price(date_ref)

In [110]:
url = '/content/drive/Shareddrives/HBong-Stock/03_datasets/price_20211205.pkl'
# df_price.to_pickle(url)
df_price = pd.read_pickle(url)

In [ ]:
!pip install -q finance-datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
missing_codes = df_[(df_.value != df_.Date)]['code'].unique()

In [86]:
def get_stock_prices(l_code, start_date=None, end_date=None):
  from multiprocessing import Pool
  # start_date = df_factor.index.levels[0].min().date().strftime('%Y%m%d')
  if start_date is None: start_date = '20190101'
  # end_date = df_factor.index.levels[0].max().date().strftime('%Y%m%d')
  # l_code = df_factor.index.levels[1].unique()
  # l_code = [] # TODO
  global get_price
  def get_price(code):
    try:
      result = (fdr.DataReader(code, start=start_date)
        .assign(code=code) )
    except Exception as e:
      print(e)
      print(f'{code} not in KRX ')
      print(f'check in {code} DELISTING')
      result = (fdr.DataReader(code, start=start_date, exchange='KRX-DELISTING')
        .assign(code=code) )
      
    return result
  with Pool(15) as pool:
    result = pool.map(get_price, l_code)
  df_price = pd.concat(result) 

  return df_price

In [87]:
df_price_deli = get_stock_prices(missing_codes).reset_index()

In [100]:
df_price = pd.concat([df_price, df_price_deli])
# df_price.columns

In [101]:
def check_min_max_date_codes():
  print(
      df_price.Date.min(), '\n',
      df_price.Date.max(), '\n',
      df_price.code.unique().__len__(), '\n',
  )
check_min_max_date_codes()
      

2011-01-03 00:00:00 
 2021-12-06 00:00:00 
 3401 



In [109]:
df_price.drop_duplicates(inplace=True)

In [105]:
df_price_deli.shape

(343992, 8)

## IV. 

1. factor
1. price
1. alphalens compute_future_return

In [ ]:
!pip install -q alphalens-reloaded

     |████████████████████████████████| 51 kB 302 kB/s 
     |████████████████████████████████| 6.3 MB 8.6 MB/s 


In [ ]:
####### factor 
def get_factor(df, order_time = '15h30m') -> pd.DataFrame:
  """make df_factor from dataframe
  Parameters
  ----------
  df: pd.DataFrame
  
  Returns
  -------
  pd.DataFrame with factor = 1
  """
  df.columns = df.columns.str.lower()
  cols_ = ['date','code','source_code']
  return (df[cols_]
  .assign(factor=1,
      datetime_to_buy=
          lambda x: pd.to_datetime(x.date) + pd.Timedelta(order_time) 
          )
  .set_index(['datetime_to_buy','code'])
  .loc[:,['factor','source_code']]
  )

In [ ]:
df_factor_1 = get_factor(df_pattern_6_20211201_) 
df_factor_2 = get_factor(df_pattern_6_20211201_, order_time='9h') 
df_factor_3 = get_factor(df_pattern_6_20211201_, order_time='0h') 

In [ ]:
df_factor_3.head(2)

factor source_code
datetime_to_buy code                      
2011-01-10      000390       1      073190
                001140       1      307180

In [111]:
###### price
df_price_ = df_price.set_index('Date')
df_open = df_price_.loc[:, ['code', 'Open']]
df_open = df_open.pivot_table(values='Open', columns='code', index=df_open.index)
df_open.index += pd.Timedelta('9h')
df_close = df_price_.loc[:,['code', 'Close']]
df_close = df_close.pivot_table(values='Close', columns='code', index=df_close.index)
df_close.index += pd.Timedelta('15h30m')
price = pd.concat([df_open, df_close], axis=0).sort_index()

In [ ]:
# price_al =\
# (df_price.reset_index()
# .rename(columns=lambda x: x.lower())
# .pivot_table(values='close',
#              index='date',
#              columns='code'))

In [112]:
price.head(2)

code,000020,000040,000050,000060,000070,000075,000080,000087,000100,000105,000120,000140,000145,000150,000155,000157,000180,000210,000215,000220,000225,000227,000230,000240,000250,000270,000300,000320,000325,000370,000390,000400,000430,000440,000480,000490,000500,000520,000540,000545,...,610025,610026,610027,610028,610029,610030,610031,610032,610033,700001,700002,700003,700004,700007,700008,700009,700010,900070,900100,900110,900120,900140,900250,900260,900270,900280,900290,900300,900310,900340,950110,950130,950140,950160,950170,950180,950190,950200,950210,950220
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-03 09:00:00,5320.0,4151.0,9160.0,10531.0,72993.0,41850.0,35400.0,NaN,27919.0,20902.0,95500.0,20250.0,14500.0,125404.0,40339.0,36941.0,1233.0,106626.0,27359.0,3711.0,1555.0,3537.0,9445.0,15895.0,2150.0,51100.0,1717.0,5870.0,7220.0,9500.0,4270.0,6246.0,4255.0,3574.0,65800.0,5100.0,25050.0,2587.0,5950.0,2775.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339.0,6830.0,2996.0,3818.0,3778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-03 15:30:00,5280.0,4172.0,9199.0,10664.0,73409.0,41625.0,35600.0,NaN,27678.0,20903.0,93600.0,20200.0,14500.0,126611.0,40669.0,36942.0,1239.0,111596.0,28477.0,3789.0,1580.0,3512.0,9142.0,16440.0,2135.0,52500.0,1753.0,5780.0,7100.0,9455.0,4285.0,6154.0,4205.0,3624.0,64000.0,5020.0,24800.0,2573.0,5990.0,2770.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1327.0,6950.0,3037.0,3956.0,4129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## wo alphalens

## 🐼pandas



In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [113]:
# df_pattern_6_20211201_.head(3) # 3mins to run ... # 1m52s
df = df_pattern_6_20211201_.copy()

##
N_points = 20
for days in range(0, N_points): # took so long
  df[f'd{days}_1'] = df.date + pd.Timedelta('9h') + days * cbday     # 해당일 시가
  df[f'd{days}_2'] = df.date + pd.Timedelta('15h30m') + days * cbday # 해당일 종가
cols_1 = ['date','code','source_code','source_date',]
cols_d_op = list(
    itertools.chain.from_iterable(
        [(f'd{i}_1',f'd{i}_2') for i in range(0,N_points)])
)
cols_all = cols_1 + cols_d_op


In [114]:
price1 = \
(price.stack()
.reset_index()
.rename(columns={0:'adj_p'})
)
price1.head()

,Date,code,adj_p
0,2011-01-03 09:00:00,000020,5320.0
1,2011-01-03 09:00:00,000040,4151.0
2,2011-01-03 09:00:00,000050,9160.0
3,2011-01-03 09:00:00,000060,10531.0
4,2011-01-03 09:00:00,000070,72993.0


In [115]:
df_ = (df
 .drop_duplicates(['code','date'])    # must to do  한 종목이 여러 소스종목과 동일한 경우가 있음 
 .melt(id_vars=cols_1, value_vars=cols_d_op)
 .sort_values(['code','variable'])
 .merge(price1, 
        left_on=['code', 'value'],
        right_on=['code','Date'], 
        how='left')
 .sort_values(['code','date','value'])
 )

In [117]:
# itook 51s
# TODO polars apply... try
sr_re = \
(df_.groupby(['date','code'], as_index=True)
.apply(lambda df: df.adj_p/ df.adj_p.iloc[1])
)

In [118]:
sr_re.values.reshape(-1, 2 * N_points).shape # (105726, 40) # (58947, 40)

(58947, 40)

df_returns.....
 - 종목명, 날짜 이후 움직임
  - d0_1 : 당일 시가
  - d0_2 : 당일 종가
  - d1_1 : 1일 뒤 시가
  - d1_2 : 1일 뒤 종가가

In [119]:
df_returns = pd.DataFrame(
    data=sr_re.values.reshape(-1, 2* N_points),
    index = sr_re.index[::40].droplevel(2),
    columns = cols_d_op
).reset_index()

In [120]:
# df_returns

기존 테이블 하고 조인

In [121]:
df_ret = (df
 .merge(df_returns,
       left_on=['date','code'],
       right_on=['date','code'],
       how='left',
       suffixes=['','_p'])
)
df_ret.shape

(105726, 85)

In [122]:
print(
    
df_ret.duplicated(['date','code']).sum(),
df.duplicated(['date','code']).sum()
)

46779 46779


In [123]:
df.shape

(105726, 45)

In [124]:
df_returns.shape

(58947, 42)

In [125]:
df_ret = (df
 .merge(df_returns,
       left_on=['date','code'],
       right_on=['date','code'],
       how='left',
       suffixes=['','_p'])
)

In [126]:
gb1 = df_ret.groupby('source_code')

In [131]:
df_ud = pd.DataFrame()

for i in range(1, 10):
  df_ud[f'up_c_{i}'] = gb1[f'd{i}_2_p'].apply(lambda x: (x>1.01).sum())
  df_ud[f'all_c_{i}'] = gb1[f'd{i}_2_p'].apply(lambda x: x.count())
  df_ud[f'ratio_c_{i}'] = df_ud[f'up_c_{i}'] / df_ud[f'all_c_{i}']

In [147]:
df_pattern_result = (df_ud
 [lambda df: df.all_c_2 > 10]
 .sort_values(['ratio_c_1','ratio_c_2','ratio_c_3','ratio_c_4'], ascending=[False, False, False, False])
 .reset_index()
)

In [148]:
table_id = f"red_lion.pattern_result_{kernel_size}_{date_ref}"
df_pattern_result.to_gbq(
    table_id, 
    project_id = 'dots-stock',
    if_exists='replace'
)

1it [00:06,  6.68s/it]


In [144]:
kernel_size=6

In [ ]:

sr_d1_2_p = gb1['d1_2_p'].apply(lambda x: x.count())

sr_d1_2_p_up.name = 'up_c'
sr_d1_2_p.name = 'all_c'

In [128]:
df_tmp = \
(pd.concat([sr_d1_2_p, sr_d1_2_p_up], axis=1)
  .assign(up_ratio = lambda df: df.up_c / df.all_c)
  [lambda df: df.all_c > 20]
)
df_tmp.sort_values(['up_ratio','all_c',], ascending=[False,False])

,all_c,up_c,up_ratio
source_code,,,
005610,73,48,0.657534
192390,35,22,0.628571
014970,87,50,0.574713
007700,42,22,0.523810
007460,86,45,0.523256
...,...,...,...
043260,53,3,0.056604
014830,25,1,0.040000
226340,28,1,0.035714


In [ ]:
gb1[[
     'd0_1_p',
     'd0_2_p',
     'd1_1_p',
     'd1_2_p',
     'd2_1_p',
     'd2_2_p',
     ]].agg(['mean','max','count'])

d0_1_p                 d0_2_p  ... d2_1_p    d2_2_p                
                 mean       max count   mean  ...  count      mean       max count
source_code                                   ...                                 
000020       1.027874  1.075446    91    1.0  ...     91  1.014272  1.339380    91
000040       0.964125  0.983853    11    1.0  ...     11  1.018283  1.072664    11
000050       0.962746  0.987315    10    1.0  ...     10  1.004573  1.040201    10
000070       0.945728  0.988372    87    1.0  ...     87  1.000219  1.160526    87
000075       0.977685  0.996219    57    1.0  ...     57  1.020221  1.204149    57
...               ...       ...   ...    ...  ...    ...       ...       ...   ...
950170       0.971211  0.986441     9    1.0  ...      9  1.016028  1.084746     9
950180       1.019775  1.020319     2    1.0  ...      2  0.950435  0.992308     2
950200       0.969880  1.000000    94    1.0  ...     94  0.990041  1.158513    94
950210       1.026956  1.045226    10    1.0  ...     10  0.984525  1.043902    10
950220       0.965012  0.993964    67    1.0  ...     67  1.006311  1.124031    67

[2080 rows x 18 columns]

In [ ]:
(gb1[[
     'd1_1_p',
     'd1_2_p',
     'd2_1_p',
     'd2_2_p',
     ]].agg(['mean','max','count'])
     [lambda df: df[('d1_1_p','count')] > 10]
     .sort_values(('d1_2_p','mean'), ascending=False)
)

d1_1_p                    d1_2_p  ... d2_1_p    d2_2_p                
                 mean       max count      mean  ...  count      mean       max count
source_code                                      ...                                 
005945       1.007174  1.084320    12  1.035371  ...     12  1.032245  1.200444    12
033180       1.003727  1.062315    15  1.029587  ...     15  1.036787  1.298300    15
111770       1.009004  1.087678    17  1.028934  ...     17  1.033519  1.315166    17
066970       1.000242  1.013489    14  1.026147  ...     14  1.044292  1.302149    14
005610       0.999777  1.054264    73  1.026133  ...     73  1.037588  1.154394    73
...               ...       ...   ...       ...  ...    ...       ...       ...   ...
123040       1.005339  1.024540    36  0.971221  ...     36  0.917343  1.096174    36
025540       1.005076  1.062824    25  0.970302  ...     25  0.910332  1.090426    25
900300       1.002435  1.023622    23  0.967760  ...     23  0.909230  1.075536    23
005720       1.000744  1.029745    42  0.959105  ...     42  0.901099  1.114286    42
019440       1.004302  1.035294    17  0.957529  ...     17  0.862792  1.076253    17

[1629 rows x 12 columns]

In [ ]:
_df_ = \
(df_ret
 [lambda df: df.source_code == '073190']
 .filter(regex='code|date|simila|_p')
)

In [ ]:
_df_.groupby('source_code')

*🔥🔥🔥*      last column number initialize
 1. reshape with numpy or torch.... not 
 2. groupby transpose
  - 🇹 groupby group get easy way 
 3. polars to increase speed

In [ ]:
gb1 = sr_re.groupby(['date', 'code'])

In [ ]:
gb1.apply(lambda df: df.shape)

date        code  
2011-01-10  000390    (34,)
            001140    (34,)
            004830    (34,)
            043090    (34,)
            043910    (34,)
                      ...  
2021-11-30  021080     (6,)
            041960     (6,)
            063760     (6,)
            304840     (6,)
            307180     (6,)
Name: adj_price, Length: 55557, dtype: object

In [ ]:
sr_re

date        code           
2011-01-10  000390  44660      0.991536
                    44708      1.000000
                    44756      0.983071
                    44793      0.996372
                    44830      0.979444
                                 ...   
2021-11-30  307180  3712983    1.000000
                    3713038    1.000000
                    3713041    0.994264
                    3713044    0.992352
                    3713047    0.978967
Name: adj_price, Length: 3776376, dtype: float64

In [ ]:
a = sr_re.to_frame()
a.columns

Index(['adj_price'], dtype='object')

In [ ]:
sr_re

date        code           
2011-01-10  000390  44660      0.991536
                    44708      1.000000
                    44756      0.983071
                    44793      0.996372
                    44830      0.979444
                                 ...   
2021-11-30  307180  3712983    1.000000
                    3713038    1.000000
                    3713041    0.994264
                    3713044    0.992352
                    3713047    0.978967
Name: adj_price, Length: 3776376, dtype: float64

In [ ]:
sr_re.groupby(['date','code']).apply(lambda df: df.pivot_table(index))

In [ ]:
(df
 [cols_all]
 .set_index(cols_1)
 .stack()
 .to_frame()
 .rename(columns={0:'date_passed'})
)

date_passed
date       code   source_code source_date                            
2011-01-10 000390 073190      2021-12-01  d0_op   2011-01-10 09:00:00
                                          d0_clos 2011-01-10 15:30:00
                                          d1_op   2011-01-11 09:00:00
                                          d1_clos 2011-01-11 15:30:00
                                          d2_op   2011-01-12 09:00:00
                                          d2_clos 2011-01-12 15:30:00
                                          d3_op   2011-01-13 09:00:00
                                          d3_clos 2011-01-13 15:30:00
           001140 307180      2021-12-01  d0_op   2011-01-10 09:00:00
                                          d0_clos 2011-01-10 15:30:00
                                          d1_op   2011-01-11 09:00:00
                                          d1_clos 2011-01-11 15:30:00
                                          d2_op   2011-01-12 09:00:00
                                          d2_clos 2011-01-12 15:30:00
                                          d3_op   2011-01-13 09:00:00
                                          d3_clos 2011-01-13 15:30:00
           004830 063080      2021-12-01  d0_op   2011-01-10 09:00:00
                                          d0_clos 2011-01-10 15:30:00
                                          d1_op   2011-01-11 09:00:00
                                          d1_clos 2011-01-11 15:30:00
                                          d2_op   2011-01-12 09:00:00
                                          d2_clos 2011-01-12 15:30:00
                                          d3_op   2011-01-13 09:00:00
                                          d3_clos 2011-01-13 15:30:00
           043090 228760      2021-12-01  d0_op   2011-01-10 09:00:00
                                          d0_clos 2011-01-10 15:30:00
                                          d1_op   2011-01-11 09:00:00
                                          d1_clos 2011-01-11 15:30:00
                                          d2_op   2011-01-12 09:00:00
                                          d2_clos 2011-01-12 15:30:00
                                          d3_op   2011-01-13 09:00:00
                                          d3_clos 2011-01-13 15:30:00
           043910 118990      2021-12-01  d0_op   2011-01-10 09:00:00
                                          d0_clos 2011-01-10 15:30:00
                                          d1_op   2011-01-11 09:00:00
                                          d1_clos 2011-01-11 15:30:00
                                          d2_op   2011-01-12 09:00:00
                                          d2_clos 2011-01-12 15:30:00
                                          d3_op   2011-01-13 09:00:00
                                          d3_clos 2011-01-13 15:30:00

In [ ]:
df.

In [ ]:
# apply pd.cbday_range 사용해서 펼쳐도 될 듯 하지만
a = \
(_df
.head(5)
 .assign(
     date_0day=
         lambda df: df.date + 0 * cbday,
     date_1day=
         lambda df: df.date + 1 * cbday,
     date_2day=
         lambda df: df.date + 2 * cbday,
     date_3day=
         lambda df: df.date + 3 * cbday,
     date_4day=
         lambda df: df.date + 4 * cbday,
     date_5day=
         lambda df: df.date + 5 * cbday,
     date_6day=
         lambda df: df.date + 6 * cbday,
     date_7day=
         lambda df: df.date + 7 * cbday
         )
)
cols_ = ['date','code','source_code','source_date',
              'date_1day',
              'date_2day',
              'date_3day',
              'date_4day',
              'date_5day',
              'date_6day',
              'date_7day',
              ]
(a
 [cols_]
 .set_index(['date','code','source_code','source_date',
              # 'date_1day',
              # 'date_2day',
              # 'date_3day',
              # 'date_4day',
              # 'date_5day',
              # 'date_6day',
              # 'date_7day',
              ])
.stack()
.name('days')
).to_frame()

##🐻 🐻 Polars

In [ ]:
from typing import List

dfp = pl.DataFrame({
    'g': [1,1,1,2,2,2,5],
    'a': [2,4,5,190, 1,4,1],
    'b': [1,3,2,1,43,3,1]
})

In [ ]:
def get_score(args: List[pl.Series])
from 


In [ ]:
!pip install polars
import polars as pl
df_pl = pl.from_pandas(df)

N_points = 20
for days in range(0, N_points): # took so long
  df_pl.with_columns([
    (pl.col('date') + pd.Timedelta('9h') + days * cbday).alias(f'd{days}_1')
    (pl.col('date') + pd.Timedelta('15h30m') + days * cbday).alias(f'd{days}_2')
          ]
  ) 

# cols_1 = ['date','code','source_code','source_date',]
# cols_d_op = list(
#     itertools.chain.from_iterable(
#         [(f'd{i}_1',f'd{i}_2') for i in range(0,N_points)])
# )
# cols_all = cols_1 + cols_d_op
# df_pl['test'] = df_pl.
price1_pl = pl.from_pandas(price1)
# polars example
df_pl2 = (df_pl.melt(id_vars=cols_1, value_vars=cols_d_op)
.sort(['code','variable'])
.join(price1_pl,
      left_on=['code', 'value'],
      right_on=['code','Date'],
      how='left')
)
(df_pl.head()
.melt(id_vars=cols_1, value_vars=cols_d_op)
)
price1_pl.head()


     |████████████████████████████████| 9.9 MB 3.8 MB/s 


## w **Alphalens**

In [ ]:
from alphalens.tears import (
    create_returns_tear_sheet,
    create_event_study_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.utils import compute_forward_returns

In [ ]:
df_factor_3.index.levels[0].min()

Timestamp('2011-01-10 00:00:00')

In [ ]:
price.index.min()

Timestamp('2011-01-03 09:00:00')

In [ ]:
(price
 .pct_change()
 .transpose() # 종목명 날짜 바꾸기
 .stack()
)

code    Date               
000020  2011-01-03 15:30:00   -0.007519
        2011-01-04 09:00:00   -0.005682
        2011-01-04 15:30:00    0.009524
        2011-01-05 09:00:00    0.000000
        2011-01-05 15:30:00   -0.011321
                                 ...   
950220  2021-11-30 15:30:00   -0.058409
        2021-12-01 09:00:00    0.000000
        2021-12-01 15:30:00    0.031016
        2021-12-02 09:00:00   -0.015560
        2021-12-02 15:30:00    0.025290
Length: 11902312, dtype: float64

In [ ]:
price.transpose().stack().pct_change()

code    Date               
000020  2011-01-03 09:00:00         NaN
        2011-01-03 15:30:00   -0.007519
        2011-01-04 09:00:00   -0.005682
        2011-01-04 15:30:00    0.009524
        2011-01-05 09:00:00    0.000000
                                 ...   
950220  2021-11-30 15:30:00   -0.058409
        2021-12-01 09:00:00    0.000000
        2021-12-01 15:30:00    0.031016
        2021-12-02 09:00:00   -0.015560
        2021-12-02 15:30:00    0.025290
Length: 11905580, dtype: float64

In [ ]:
df_factor.reset_index()

,datetime_to_buy,code,factor,source_code
0,2011-01-10 15:30:00,000390,1,073190
1,2011-01-10 15:30:00,001140,1,307180
2,2011-01-10 15:30:00,004830,1,063080
3,2011-01-10 15:30:00,043090,1,228760
4,2011-01-10 15:30:00,043910,1,118990
...,...,...,...,...
105721,2021-11-30 15:30:00,021080,1,245620
105722,2021-11-30 15:30:00,041960,1,214870
105723,2021-11-30 15:30:00,063760,1,302440
105724,2021-11-30 15:30:00,304840,1,317240


In [ ]:
df_factor = df_factor.tz_localize('UTC', level=0)

In [ ]:
price_al = price_al.tz_localize('UTC')

In [ ]:
price1 = price.tz_localize('UTC')

In [ ]:
# compute_forward_returns(df_factor, price1)

ValueError: ignored

In [ ]:
df_factor.index

MultiIndex([('2011-01-10 15:30:00', '000390'),
            ('2011-01-10 15:30:00', '001140'),
            ('2011-01-10 15:30:00', '004830'),
            ('2011-01-10 15:30:00', '043090'),
            ('2011-01-10 15:30:00', '043910'),
            ('2011-01-10 15:30:00', '054340'),
            ('2011-01-10 15:30:00', '057050'),
            ('2011-01-10 15:30:00', '071530'),
            ('2011-01-10 15:30:00', '099320'),
            ('2011-01-11 15:30:00', '001515'),
            ...
            ('2021-11-29 15:30:00', '169330'),
            ('2021-11-29 15:30:00', '190650'),
            ('2021-11-29 15:30:00', '214680'),
            ('2021-11-29 15:30:00', '255440'),
            ('2021-11-29 15:30:00', '281740'),
            ('2021-11-30 15:30:00', '021080'),
            ('2021-11-30 15:30:00', '041960'),
            ('2021-11-30 15:30:00', '063760'),
            ('2021-11-30 15:30:00', '304840'),
            ('2021-11-30 15:30:00', '307180')],
           names=['datetime_to_buy', 'code'

In [ ]:
factor_data = get_clean_factor_and_forward_returns(
    df_factor,
    price,
    quantiles=None,
    periods = [2,4,6] , # 0 은 그 시간, 1: 그다음 아침 시가
    max_loss = 0.7,
    bins = 1
)

ValueError: ignored

In [ ]:
out = df.groupby("foo").pivot(pivot_column="bar", values_column="N").first()
a = df_price.set_index('Date').loc[:,['code', 'Open']]
a.pivot_table(values='Open', columns='code', index=a.index)

In [ ]:
df_pl = pl.from_pandas(df_price.reset_index())

In [ ]:
# df_pl.groupby('Date').pivot(pivot_column='code', values_column='Open').first()

In [ ]:
!pip install polars

     |████████████████████████████████| 9.9 MB 6.4 MB/s 


In [ ]:
import polars as pl

In [ ]:
data_pl = pl.from_pandas(df_price)

In [ ]:
data_pl.head()

code,Date,Open,High,Low,Close,Volume,Change
str,datetime,i64,i64,i64,i64,i64,f64
"""098120""",2014-09-23 00:00:00,10600,10850,10100,10400,106419,-2.3474178403755874e-2
"""138920""",2014-09-23 00:00:00,10965,11015,10870,11010,1367,-2.7173913043477826e-3
"""078930""",2014-09-23 00:00:00,41600,42100,41250,41700,291818,-5.959475566150174e-3
"""079160""",2014-09-23 00:00:00,48982,48982,47893,48257,139946,-1.2987809866644828e-2
"""042500""",2014-09-23 00:00:00,4980,5010,4800,4925,834746,-1.1044176706827336e-2


In [ ]:
data_pl.groupby('code')['Close'].

# XXXXXXXXXXXXX TESTING


In [ ]:
__df = (df
 .drop_duplicates(['code','date'])    # must to do  한 종목이 여러 소스종목과 동일한 경우가 있음 
 .melt(id_vars=cols_1, value_vars=cols_d_op)
 .sort_values(['code','variable']
 ))

In [ ]:
__df.loc[0,'value']

Timestamp('2011-01-10 09:00:00')

In [ ]:
price1.loc[0,'Date']

Timestamp('2011-01-03 09:00:00')

In [ ]:
df_ = (df
 .drop_duplicates(['code','date'])    # must to do  한 종목이 여러 소스종목과 동일한 경우가 있음 
 .melt(id_vars=cols_1, value_vars=cols_d_op)
 .sort_values(['code','variable'])
 .merge(price1, 
        left_on=['code', 'value'],
        right_on=['code','Date'], 
        how='left')
 .sort_values(['code','date','value'])
 )

In [ ]:
df_

,date,code,source_code,source_date,variable,value,Date,adj_p
0,2011-03-15,000020,317240,2021-12-01,d0_1,2011-03-15 09:00:00,2011-03-15 09:00:00,4290.0
34,2011-03-15,000020,317240,2021-12-01,d0_2,2011-03-15 15:30:00,2011-03-15 15:30:00,4070.0
748,2011-03-15,000020,317240,2021-12-01,d1_1,2011-03-16 09:00:00,2011-03-16 09:00:00,4040.0
782,2011-03-15,000020,317240,2021-12-01,d1_2,2011-03-16 15:30:00,2011-03-16 15:30:00,4140.0
816,2011-03-15,000020,317240,2021-12-01,d2_1,2011-03-17 09:00:00,2011-03-17 09:00:00,4065.0
...,...,...,...,...,...,...,...,...
2357835,2021-06-10,950220,023460,2021-12-01,d17_2,2021-07-05 15:30:00,2021-07-05 15:30:00,11750.0
2357837,2021-06-10,950220,023460,2021-12-01,d18_1,2021-07-06 09:00:00,2021-07-06 09:00:00,11850.0
2357839,2021-06-10,950220,023460,2021-12-01,d18_2,2021-07-06 15:30:00,2021-07-06 15:30:00,11700.0
2357841,2021-06-10,950220,023460,2021-12-01,d19_1,2021-07-07 09:00:00,2021-07-07 09:00:00,11650.0


In [ ]:
(price1
 [lambda df: df.Date == pd.Timestamp('2014-12-02T09')]
#  [lambda df: df.code == '000030']
)

In [ ]:
fdr.DataReader('002270')

,Open,High,Low,Close,Volume,Change
Date,,,,,,
1997-11-04,25793,25793,25793,25794,2990,NaN
1997-11-05,25391,26599,25391,26358,6750,0.021866
1997-11-06,24585,26196,24585,26197,200,-0.006108
1997-11-07,24101,25713,24101,25713,2020,-0.018475
1997-11-08,24907,24907,24907,24907,300,-0.031346
...,...,...,...,...,...,...
2021-11-30,344000,344000,318000,318500,6971,-0.070073
2021-12-01,321000,326500,320000,325000,743,0.020408
2021-12-02,322000,339000,322000,338500,1469,0.041538
